In [57]:
import numpy as np, pandas as pd, sklearn, json
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [14]:
df1 = pd.read_csv(r"C:\Users\pc\Downloads\customer_nodes.csv")
df1

,Unnamed: 0,customer_id,region_id,node_id,start_date,end_date
0,0,1,3,4,2020-01-02,2020-01-03
1,1,2,3,5,2020-01-03,2020-01-17
2,2,3,5,4,2020-01-27,2020-02-18
3,3,4,5,4,2020-01-07,2020-01-19
4,4,5,3,3,2020-01-15,2020-01-23
...,...,...,...,...,...,...
3495,3495,496,3,4,2020-02-25,9999-12-31
3496,3496,497,5,4,2020-05-27,9999-12-31
3497,3497,498,1,2,2020-04-05,9999-12-31
3498,3498,499,5,1,2020-02-03,9999-12-31


In [23]:
df2 = pd.read_csv(r"C:\Users\pc\Downloads\regions.csv")
df2

,Unnamed: 0,region_id,region_name
0,0,1,Australia
1,1,2,America
2,2,3,Africa
3,3,4,Asia
4,4,5,Europe


In [24]:
df3 = pd.read_csv(r"C:\Users\pc\Downloads\customer_transactions.csv")
df3

,Unnamed: 0,customer_id,txn_date,txn_type,txn_amount
0,0,429,2020-01-21,deposit,82
1,1,155,2020-01-10,deposit,712
2,2,398,2020-01-01,deposit,196
3,3,255,2020-01-14,deposit,563
4,4,185,2020-01-29,deposit,626
...,...,...,...,...,...
5863,5863,189,2020-02-03,withdrawal,870
5864,5864,189,2020-03-22,purchase,718
5865,5865,189,2020-02-06,purchase,393
5866,5866,189,2020-01-22,deposit,302


### 1.How many unique nodes are there on the Data Bank system?

In [9]:
# It Calculates the number of unique values in the 'node_id' column of the DataFrame df1 and store the result in the variable 'a'.
a = df1['node_id'].nunique()
a

5

### 2. What is the number of nodes per region?

In [15]:
# Group the DataFrame df1 by the 'region_id' column and calculate the count of unique values in the 'node_id' column for each group. 
b = df1.groupby('region_id')['node_id'].count()

region_id
1    770
2    735
3    714
4    665
5    616
Name: node_id, dtype: int64

### 3. How many customers are allocated to each region

In [59]:
# Group the DataFrame df1 by the 'region_id' column and calculate the count of unique customer_ids for each region
customers = df1.groupby('region_id')['customer_id'].count()
customers

region_id
1    770
2    735
3    714
4    665
5    616
Name: customer_id, dtype: int64

### 4. How many days on average are customers reallocated to a different node?

In [60]:
# 'coerce' parameter is used to handle errors that may arise during the conversion of a column to a datetime format.
df1['start_date'] = pd.to_datetime(df1['start_date'],errors = 'coerce')
# Convert the 'start_date' and 'end_date' columns in the DataFrame df1 to datetime format
df1['end_date'] = pd.to_datetime(df1['end_date'],errors = 'coerce')

In [61]:
df1['average_days'] = df1['end_date'] - df1['start_date']
# Calculate the difference between 'end_date' and 'start_date' columns to obtain the duration in days,
df1['average_days']
average_days_reallocation = df1.groupby('customer_id')['average_days'].mean()
# Calculate the mean of 'average_days' for each customer by grouping the DataFrame df1 by 'customer_id'.
average_days_reallocation

customer_id
1     11 days 12:00:00
2     10 days 20:00:00
3     13 days 20:00:00
4     13 days 08:00:00
5      7 days 12:00:00
            ...       
496    7 days 08:00:00
497   18 days 16:00:00
498   11 days 20:00:00
499    4 days 08:00:00
500   14 days 00:00:00
Name: average_days, Length: 500, dtype: timedelta64[ns]

### 5. What is the median, 80th and 95th percentile for this same reallocation days metric for each region?

In [40]:
df1.groupby('region_id')['average_days'].agg(['median', lambda x : x.quantile(0.8), lambda x : x.quantile(0.95)])
# Group the DataFrame df1 by 'region_id' and calculate specific statistics on the 'average_days' column.
# - 'median': Calculate the median of the 'average_days'.
# - lambda x: x.quantile(0.8): Calculate the 80th percentile (quantile) of the 'average_days'.
# - lambda x: x.quantile(0.95): Calculate the 95th percentile (quantile) of the 'average_days'.

,median,<lambda_0>,<lambda_1>
region_id,,,
1,12 days,23 days,28 days
2,12 days,23 days,28 days
3,12 days,24 days,28 days
4,12 days,23 days,28 days
5,12 days,24 days,28 days


In [46]:
df = pd.merge(df1, df3, on = 'customer_id') # it merges on df1 column and df3 colum on 'customer_id'
df = df.drop(['Unnamed: 0_x','Unnamed: 0_y'],axis = 1) # it drops the unnamed 0_x,unnamed 0_y column on column wise
df

,customer_id,region_id,node_id,start_date,end_date,average_days,txn_date,txn_type,txn_amount
0,1,3,4,2020-01-02,2020-01-03,1 days,2020-01-02,deposit,312
1,1,3,4,2020-01-02,2020-01-03,1 days,2020-03-05,purchase,612
2,1,3,4,2020-01-02,2020-01-03,1 days,2020-03-17,deposit,324
3,1,3,4,2020-01-02,2020-01-03,1 days,2020-03-19,purchase,664
4,1,3,4,2020-01-04,2020-01-14,10 days,2020-01-02,deposit,312
...,...,...,...,...,...,...,...,...,...
41071,500,2,2,2020-04-15,NaT,NaT,2020-01-27,deposit,214
41072,500,2,2,2020-04-15,NaT,NaT,2020-03-01,purchase,929
41073,500,2,2,2020-04-15,NaT,NaT,2020-01-30,deposit,922
41074,500,2,2,2020-04-15,NaT,NaT,2020-01-18,deposit,308


### 6. What is the unique count and total amount for each transaction type?

In [47]:
# it calculates the no.of unique values in the txt_type column of the df(dataframe)
df['txn_type'].nunique()

3

In [48]:
df.groupby('txn_type')['txn_amount'].sum()
# it calculates the sum by doing groupby on the txn_type and the txt_amount

txn_type
deposit       9514176
purchase      5645759
withdrawal    5551021
Name: txn_amount, dtype: int64

### 7. What is the average total historical deposit counts and amounts for all customers?

In [50]:
#df

In [51]:
# Filter the DataFrame df to include only rows where the 'txn_type' column is equal to 'deposit'.
df = df[df['txn_type']=='deposit']
df

,customer_id,region_id,node_id,start_date,end_date,average_days,txn_date,txn_type,txn_amount
0,1,3,4,2020-01-02,2020-01-03,1 days,2020-01-02,deposit,312
2,1,3,4,2020-01-02,2020-01-03,1 days,2020-03-17,deposit,324
4,1,3,4,2020-01-04,2020-01-14,10 days,2020-01-02,deposit,312
6,1,3,4,2020-01-04,2020-01-14,10 days,2020-03-17,deposit,324
8,1,3,2,2020-01-15,2020-01-16,1 days,2020-01-02,deposit,312
...,...,...,...,...,...,...,...,...,...
41068,500,2,2,2020-04-15,NaT,NaT,2020-03-02,deposit,862
41070,500,2,2,2020-04-15,NaT,NaT,2020-02-23,deposit,721
41071,500,2,2,2020-04-15,NaT,NaT,2020-01-27,deposit,214
41073,500,2,2,2020-04-15,NaT,NaT,2020-01-30,deposit,922


In [54]:
x = df.groupby('customer_id')['txn_amount'].count().mean()
# To calculate the count and mean by doing the groupby on customer_id and txn_amount columns 
y = df.groupby('customer_id')['txn_amount'].sum().mean()
# To calculate the sum and mean value on customer_id and txn_amount columns
x,y

(37.394, 19028.352)

### 8. What is the closing balance for each customer at the end of the month?

In [58]:
df['amount'] = df.groupby('customer_id')['txn_amount'].cumsum()
# Calculates the cumulative sum of 'txn_amount' for each customer using the 'groupby' and 'cumsum' functions.
closing_balances = df.groupby(['customer_id', df['end_date'].dt.to_period("M")])['amount'].last()
# Group the DataFrame df by both 'customer_id' and the monthly period of 'end_date',
closing_balances
# then extract the last cumulative sum value for each group as closing balances.

customer_id  end_date
1            2020-01      2544
             2020-02      3180
             2020-03      3816
2            2020-01       610
             2020-02      1830
                         ...  
499          2020-02     27102
500          2020-01      6047
             2020-02     18141
             2020-03     30235
             2020-04     36282
Name: amount, Length: 1784, dtype: int64